# Notebook 03: Phase 0 - So sánh Random Split vs Time-Series Split

## Mục tiêu
Chứng minh **"Bẫy nội suy" (Interpolation Trap)** - sự khác biệt giữa Random Split va Time-Series Split.

## Cấu trúc thí nghiệm
- **2 phương pháp chia dữ liệu**: Random Split (80/20) vs Time-Series Split (2001-2014 train, 2015-2019 test)
- **2 cách tiếp cận dữ liệu**: Pooled Data vs Panel Data
- **3 thuật toán**: Linear Regression, SVR, XGBoost

**Tổng cộng: 2 x 2 x 3 = 12 thí nghiệm**

## Định nghĩa
- **Pooled Data**: Coi moi quan sát la độc lập, KHONG sử dụng Entity encoding va Lag features
- **Panel Data**: Su dung Fixed Effects (Entity encoding) va Lag features để học xu hướng lịch sử

## XU LY DATA LEAKAGE
**Vấn đề**: CO2_lag1 (CO2 năm trước) có thể gây Data Leakage khi dùng Random Split:
- CO2_lag1 của năm 2016 = CO2 thuc của năm 2015
- Nếu Random Split đưa 2015 vao test set va 2016 vao train set
- Model se "biết" CO2 cua 2015 thông qua CO2_lag1 cua 2016 -> LEAKAGE!

**Giải pháp**:
- **Random Split + Panel**: LOAI BO CO2_lag1 (remove_target_lag=True)
- **Time-Series Split + Panel**: GIỮ CO2_lag1 (không có leakage vì train < test theo thời gian)

## Kết quả mong đợi
- Random Split: R2 cao do nội suy (interpolation)
- Time-Series Split: R2 thấp hơn cho XGBoost va SVR (khong ngoại suy tốt)
- Linear Regression: Ổn định cả hai cach chia (ngoại suy tốt)

## 1. Thiết lập môi trường va tải dữ liệu

In [ ]:
# 1.1 Import thư viện
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import sys
import os

# Thêm src vào path
sys.path.append(os.path.abspath(os.path.join("../src")))

# Cấu hình
pd.set_option("display.max_columns", 30)
RANDOM_STATE = 42
TEST_SIZE = 0.2
TARGET = "Value_co2_emissions_kt_by_country"
TRAIN_END_YEAR = 2014  # Time-Series Split: Train 2001-2014, Test 2015-2019

print("Thư viện đã được import thành công.")

In [ ]:
# 1.2 Tải dữ liệu da tiền xử lý cho từng model
df_lr = pd.read_csv("../data/processed/lr_final_prep.csv")
df_svr = pd.read_csv("../data/processed/svr_final_prep.csv")
df_xgb = pd.read_csv("../data/processed/xgb_final_prep.csv")

print(f"LR data shape: {df_lr.shape}")
print(f"SVR data shape: {df_svr.shape}")
print(f"XGBoost data shape: {df_xgb.shape}")

In [ ]:
# 1.3 Định nghĩa cac hàm hỗ trợ

def evaluate_model(y_true, y_pred, model_năme):
    """Đánh giá model va trả về dict kết quả"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {"Model": model_năme, "RMSE": rmse, "MAE": mae, "R2": r2}

def get_pooled_features(df, target, has_entity_onehot=True):
    """
    Pooled Data: Loại bỏ lag features va Entity encoding.
    Coi moi quan sát la độc lập, không xét yếu tố quốc gia/thời gian.
    """
    df_copy = df.copy()
    
    # Xác định cac cột cần loại bỏ
    lag_cols = [c for c in df_copy.columns if "lag" in c.lower()]
    
    if has_entity_onehot:
        entity_cols = [c for c in df_copy.columns if c.startswith("Entity_")]
    else:
        entity_cols = ["Entity"] if "Entity" in df_copy.columns else []
    
    year_cols = ["Year"] if "Year" in df_copy.columns else []
    
    drop_cols = lag_cols + entity_cols + year_cols + [target]
    drop_cols = [c for c in drop_cols if c in df_copy.columns]
    
    X = df_copy.drop(columns=drop_cols)
    y = df_copy[target]
    
    return X, y

def get_panel_features(df, target, has_entity_onehot=True, remove_target_lag=False):
    """
    Panel Data: Giữ lại lag features va Entity encoding.
    Su dung Fixed Effects (Entity) va Lag Features để học xu hướng lịch sử.
    
    QUAN TRONG - DATA LEAKAGE:
    - remove_target_lag=True: Loại bỏ CO2_lag1 (dung cho Random Split để tránh leakage)
    - remove_target_lag=False: Giu CO2_lag1 (dung cho Time-Series Split - không có leakage)
    
    Lý do: Với Random Split, CO2_lag1 của năm 2016 = CO2 thực của 2015.
    Nếu 2015 năm trong test set va 2016 năm trong train set -> Model biết CO2 cua 2015
    thông qua CO2_lag1 cua 2016 -> DATA LEAKAGE!
    """
    df_copy = df.copy()
    
    year_cols = ["Year"] if "Year" in df_copy.columns else []
    drop_cols = year_cols + [target]
    
    # Loại bỏ CO2_lag1 nếu cần (tránh data leakage voi Random Split)
    if remove_target_lag:
        target_lag_cols = [c for c in df_copy.columns if "co2" in c.lower() and "lag" in c.lower()]
        drop_cols = drop_cols + target_lag_cols
        if target_lag_cols:
            print(f"  [Panel] Loại bỏ target lag columns để tránh data leakage: {target_lag_cols}")
    
    drop_cols = [c for c in drop_cols if c in df_copy.columns]
    
    X = df_copy.drop(columns=drop_cols)
    y = df_copy[target]
    
    return X, y

print("Cac hàm hỗ trợ đã được định nghĩa.")
print("\nLƯU Ý VỀ DATA LEAKAGE:")
print("- Random Split + Panel Data: PHAI loại bỏ CO2_lag1 (remove_target_lag=True)")
print("- Time-Series Split + Panel Data: GIỮ CO2_lag1 (remove_target_lag=False)")

In [ ]:
# 1.4 Hàm chia dữ liệu theo Time-Series

def time_series_split(df, train_end_year=2014):
    """
    Chia dữ liệu theo thời gian:
    - Train: 2001-2014 (14 năm)
    - Test: 2015-2019 (5 năm)
    """
    train_mask = df["Year"] <= train_end_year
    test_mask = df["Year"] > train_end_year
    
    return df[train_mask].copy(), df[test_mask].copy()

print("Hàm time_series_split đã được định nghĩa.")

---
## 2. RANDOM SPLIT (80/20)

**Cảnh báo**: Random Split trộn lẫn cac năm -> Model "nhìn thấy tương lai" khi train -> Data Leakage!

### 2.1 Random Split + Pooled Data

In [ ]:
# 2.1.1 Linear Regression - Random Split + Pooled
print("=== LINEAR REGRESSION - Random Split + Pooled ===")

X, y = get_pooled_features(df_lr, TARGET, has_entity_onehot=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_rs_pooled_lr = evaluate_model(y_test, y_pred, "LR")
print(f"R2: {result_rs_pooled_lr['R2']:.4f}")

In [ ]:
# 2.1.2 SVR - Random Split + Pooled
print("=== SVR - Random Split + Pooled ===")

X, y = get_pooled_features(df_svr, TARGET, has_entity_onehot=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

# SVR can scale target
model = TransformedTargetRegressor(
    regressor=SVR(kernel="rbf", C=10, epsilon=0.1),
    transformer=StandardScaler()
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_rs_pooled_svr = evaluate_model(y_test, y_pred, "SVR")
print(f"R2: {result_rs_pooled_svr['R2']:.4f}")

In [ ]:
# 2.1.3 XGBoost - Random Split + Pooled
print("=== XGBOOST - Random Split + Pooled ===")

X, y = get_pooled_features(df_xgb, TARGET, has_entity_onehot=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)

result_rs_pooled_xgb = evaluate_model(y_test, y_pred, "XGBoost")
print(f"R2: {result_rs_pooled_xgb['R2']:.4f}")

### 2.2 Random Split + Panel Data

In [ ]:
# 2.2.1 Linear Regression - Random Split + Panel
print("=== LINEAR REGRESSION - Random Split + Panel ===")
print("[!] Loại bỏ CO2_lag1 để tránh Data Leakage")

X, y = get_panel_features(df_lr, TARGET, has_entity_onehot=True, remove_target_lag=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (bao gồm Entity One-Hot, KHONG co CO2_lag1)")

model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_rs_panel_lr = evaluate_model(y_test, y_pred, "LR")
print(f"R2: {result_rs_panel_lr['R2']:.4f}")

In [ ]:
# 2.2.2 SVR - Random Split + Panel
print("=== SVR - Random Split + Panel ===")
print("[!] Loại bỏ CO2_lag1 để tránh Data Leakage")

X, y = get_panel_features(df_svr, TARGET, has_entity_onehot=True, remove_target_lag=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (KHONG co CO2_lag1)")

model = TransformedTargetRegressor(
    regressor=SVR(kernel="rbf", C=10, epsilon=0.1),
    transformer=StandardScaler()
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_rs_panel_svr = evaluate_model(y_test, y_pred, "SVR")
print(f"R2: {result_rs_panel_svr['R2']:.4f}")

In [ ]:
# 2.2.3 XGBoost - Random Split + Panel
print("=== XGBOOST - Random Split + Panel ===")
print("[!] Loại bỏ CO2_lag1 để tránh Data Leakage")

X, y = get_panel_features(df_xgb, TARGET, has_entity_onehot=False, remove_target_lag=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (KHONG co CO2_lag1)")

model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)

result_rs_panel_xgb = evaluate_model(y_test, y_pred, "XGBoost")
print(f"R2: {result_rs_panel_xgb['R2']:.4f}")

---
## 3. TIME-SERIES SPLIT (Train: 2001-2014, Test: 2015-2019)

**Phương pháp đúng**: Model KHÔNG BAO GIỜ nhìn thấy dữ liệu tương lai khi train.

### 3.1 Time-Series Split + Pooled Data

In [ ]:
# 3.1.1 Linear Regression - Time-Series Split + Pooled
print("=== LINEAR REGRESSION - Time-Series Split + Pooled ===")

df_train, df_test = time_series_split(df_lr, TRAIN_END_YEAR)

X_train, y_train = get_pooled_features(df_train, TARGET, has_entity_onehot=True)
X_test, y_test = get_pooled_features(df_test, TARGET, has_entity_onehot=True)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_ts_pooled_lr = evaluate_model(y_test, y_pred, "LR")
print(f"R2: {result_ts_pooled_lr['R2']:.4f}")

In [ ]:
# 3.1.2 SVR - Time-Series Split + Pooled
print("=== SVR - Time-Series Split + Pooled ===")

df_train, df_test = time_series_split(df_svr, TRAIN_END_YEAR)

X_train, y_train = get_pooled_features(df_train, TARGET, has_entity_onehot=True)
X_test, y_test = get_pooled_features(df_test, TARGET, has_entity_onehot=True)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

model = TransformedTargetRegressor(
    regressor=SVR(kernel="rbf", C=10, epsilon=0.1),
    transformer=StandardScaler()
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_ts_pooled_svr = evaluate_model(y_test, y_pred, "SVR")
print(f"R2: {result_ts_pooled_svr['R2']:.4f}")

In [ ]:
# 3.1.3 XGBoost - Time-Series Split + Pooled
print("=== XGBOOST - Time-Series Split + Pooled ===")

df_train, df_test = time_series_split(df_xgb, TRAIN_END_YEAR)

X_train, y_train = get_pooled_features(df_train, TARGET, has_entity_onehot=False)
X_test, y_test = get_pooled_features(df_test, TARGET, has_entity_onehot=False)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]}")

model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)

result_ts_pooled_xgb = evaluate_model(y_test, y_pred, "XGBoost")
print(f"R2: {result_ts_pooled_xgb['R2']:.4f}")

### 3.2 Time-Series Split + Panel Data

In [ ]:
# 3.2.1 Linear Regression - Time-Series Split + Panel
print("=== LINEAR REGRESSION - Time-Series Split + Panel ===")
print("[OK] GIỮ CO2_lag1 vi Time-Series Split không có Data Leakage")

df_train, df_test = time_series_split(df_lr, TRAIN_END_YEAR)

X_train, y_train = get_panel_features(df_train, TARGET, has_entity_onehot=True, remove_target_lag=False)
X_test, y_test = get_panel_features(df_test, TARGET, has_entity_onehot=True, remove_target_lag=False)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (bao gồm Entity One-Hot va CO2_lag1)")

model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_ts_panel_lr = evaluate_model(y_test, y_pred, "LR")
print(f"R2: {result_ts_panel_lr['R2']:.4f}")

In [ ]:
# 3.2.2 SVR - Time-Series Split + Panel
print("=== SVR - Time-Series Split + Panel ===")
print("[OK] GIỮ CO2_lag1 vi Time-Series Split không có Data Leakage")

df_train, df_test = time_series_split(df_svr, TRAIN_END_YEAR)

X_train, y_train = get_panel_features(df_train, TARGET, has_entity_onehot=True, remove_target_lag=False)
X_test, y_test = get_panel_features(df_test, TARGET, has_entity_onehot=True, remove_target_lag=False)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (bao gồm CO2_lag1)")

model = TransformedTargetRegressor(
    regressor=SVR(kernel="rbf", C=10, epsilon=0.1),
    transformer=StandardScaler()
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

result_ts_panel_svr = evaluate_model(y_test, y_pred, "SVR")
print(f"R2: {result_ts_panel_svr['R2']:.4f}")

In [ ]:
# 3.2.3 XGBoost - Time-Series Split + Panel
print("=== XGBOOST - Time-Series Split + Panel ===")
print("[OK] GIỮ CO2_lag1 vi Time-Series Split không có Data Leakage")

df_train, df_test = time_series_split(df_xgb, TRAIN_END_YEAR)

X_train, y_train = get_panel_features(df_train, TARGET, has_entity_onehot=False, remove_target_lag=False)
X_test, y_test = get_panel_features(df_test, TARGET, has_entity_onehot=False, remove_target_lag=False)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"Số lượng features: {X_train.shape[1]} (bao gồm CO2_lag1)")

model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)

result_ts_panel_xgb = evaluate_model(y_test, y_pred, "XGBoost")
print(f"R2: {result_ts_panel_xgb['R2']:.4f}")

---
## 4. Tong hop va So sánh Kết quả

In [ ]:
# 4.1 Tạo bảng tổng hợp kết quả

# Random Split results
random_split_results = [
    {"Split": "Random", "Approach": "Pooled", **result_rs_pooled_lr},
    {"Split": "Random", "Approach": "Pooled", **result_rs_pooled_svr},
    {"Split": "Random", "Approach": "Pooled", **result_rs_pooled_xgb},
    {"Split": "Random", "Approach": "Panel", **result_rs_panel_lr},
    {"Split": "Random", "Approach": "Panel", **result_rs_panel_svr},
    {"Split": "Random", "Approach": "Panel", **result_rs_panel_xgb},
]

# Time-Series Split results
timeseries_split_results = [
    {"Split": "TimeSeries", "Approach": "Pooled", **result_ts_pooled_lr},
    {"Split": "TimeSeries", "Approach": "Pooled", **result_ts_pooled_svr},
    {"Split": "TimeSeries", "Approach": "Pooled", **result_ts_pooled_xgb},
    {"Split": "TimeSeries", "Approach": "Panel", **result_ts_panel_lr},
    {"Split": "TimeSeries", "Approach": "Panel", **result_ts_panel_svr},
    {"Split": "TimeSeries", "Approach": "Panel", **result_ts_panel_xgb},
]

# Combine all results
all_results = random_split_results + timeseries_split_results
df_results = pd.DataFrame(all_results)

# Reorder columns
df_results = df_results[["Split", "Approach", "Model", "R2", "RMSE", "MAE"]]

print("\n" + "="*80)
print("BANG TONG HOP KET QUA - 12 THI NGHIEM")
print("="*80)
print(df_results.to_string(index=False))

In [ ]:
# 4.2 Phân tích su sụt giảm R2 giữa Random Split va Time-Series Split

print("\n" + "="*80)
print("PHAN TICH SU SUY GIAM R2: Random Split -> Time-Series Split")
print("="*80)

# Pooled Data comparison
print("\n--- POOLED DATA ---")
print(f"LR:      Random R2={result_rs_pooled_lr['R2']:.4f} -> TS R2={result_ts_pooled_lr['R2']:.4f} | Giam: {(result_rs_pooled_lr['R2']-result_ts_pooled_lr['R2'])*100:.1f}%")
print(f"SVR:     Random R2={result_rs_pooled_svr['R2']:.4f} -> TS R2={result_ts_pooled_svr['R2']:.4f} | Giam: {(result_rs_pooled_svr['R2']-result_ts_pooled_svr['R2'])*100:.1f}%")
print(f"XGBoost: Random R2={result_rs_pooled_xgb['R2']:.4f} -> TS R2={result_ts_pooled_xgb['R2']:.4f} | Giam: {(result_rs_pooled_xgb['R2']-result_ts_pooled_xgb['R2'])*100:.1f}%")

# Panel Data comparison
print("\n--- PANEL DATA ---")
print(f"LR:      Random R2={result_rs_panel_lr['R2']:.4f} -> TS R2={result_ts_panel_lr['R2']:.4f} | Giam: {(result_rs_panel_lr['R2']-result_ts_panel_lr['R2'])*100:.1f}%")
print(f"SVR:     Random R2={result_rs_panel_svr['R2']:.4f} -> TS R2={result_ts_panel_svr['R2']:.4f} | Giam: {(result_rs_panel_svr['R2']-result_ts_panel_svr['R2'])*100:.1f}%")
print(f"XGBoost: Random R2={result_rs_panel_xgb['R2']:.4f} -> TS R2={result_ts_panel_xgb['R2']:.4f} | Giam: {(result_rs_panel_xgb['R2']-result_ts_panel_xgb['R2'])*100:.1f}%")

In [ ]:
# 4.3 Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pooled Data comparison
models = ["LR", "SVR", "XGBoost"]
random_pooled = [result_rs_pooled_lr["R2"], result_rs_pooled_svr["R2"], result_rs_pooled_xgb["R2"]]
ts_pooled = [result_ts_pooled_lr["R2"], result_ts_pooled_svr["R2"], result_ts_pooled_xgb["R2"]]

x = np.arange(len(models))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, random_pooled, width, label="Random Split", color="steelblue")
bars2 = ax1.bar(x + width/2, ts_pooled, width, label="Time-Series Split", color="coral")
ax1.set_ylabel("R2 Score")
ax1.set_title("POOLED DATA: Random vs Time-Series Split")
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.set_ylim(0, 1.1)
ax1.axhline(y=0.9, color="green", linestyle="--", alpha=0.5, label="R2=0.9")

# Panel Data comparison
random_panel = [result_rs_panel_lr["R2"], result_rs_panel_svr["R2"], result_rs_panel_xgb["R2"]]
ts_panel = [result_ts_panel_lr["R2"], result_ts_panel_svr["R2"], result_ts_panel_xgb["R2"]]

ax2 = axes[1]
bars3 = ax2.bar(x - width/2, random_panel, width, label="Random Split", color="steelblue")
bars4 = ax2.bar(x + width/2, ts_panel, width, label="Time-Series Split", color="coral")
ax2.set_ylabel("R2 Score")
ax2.set_title("PANEL DATA: Random vs Time-Series Split")
ax2.set_xticks(x)
ax2.set_xticklabels(models)
ax2.legend()
ax2.set_ylim(0, 1.1)
ax2.axhline(y=0.9, color="green", linestyle="--", alpha=0.5, label="R2=0.9")

plt.tight_layout()
plt.savefig("../reports/figures/random_vs_timeseries_comparison.png", dpi=150, bbox_inches="tight")
plt.show()

print("\nBiểu đồ đã được lưu tại: reports/figures/random_vs_timeseries_comparison.png")

---
## 5. Kết luận

### 5.1 Xử lý Data Leakage

**Vấn đề CO2_lag1**:
- CO2_lag1 của năm t = CO2 thuc của năm t-1
- Với Random Split: Nếu năm t-1 năm trong test set, model se "biết" giá trị test thông qua CO2_lag1
- **Giải pháp**: Loại bỏ CO2_lag1 khi dùng Random Split + Panel Data

### 5.2 Phân tích kết quả chi tiet

#### A. POOLED DATA (Khong dung Lag features va Entity encoding)

| Thuat toan | Random R2 | TimeSeries R2 | Giam | Nhận xét |
|------------|-----------|---------------|------|----------|
| LR | 0.8543 | 0.8310 | 2.3% | Ổn định, ngoại suy tốt |
| SVR | 0.9255 | 0.9028 | 2.3% | Ổn định, RBF kernel van hoat dong |
| XGBoost | 0.9765 | 0.7369 | **24.0%** | **SUY GIAM MẠNH** |

**Nhận xét Pooled Data**:
- LR va SVR: Suy giảm nhe (~2.3%) -> **Ổn định** khi chuyển từ nội suy sang ngoại suy
- XGBoost: Suy giảm **24%** -> **Khong ngoại suy tốt**
- Lý do: Pooled Data không có thông tin lịch sử (lag), nen model phụ thuộc hoàn toàn vao features hiện tại

#### B. PANEL DATA (Co Lag features va Entity encoding, DA LOAI BO CO2_lag1 cho Random Split)

| Thuat toan | Random R2 | TimeSeries R2 | Giam | Nhận xét |
|------------|-----------|---------------|------|----------|
| LR | 0.9959 | 0.9934 | 0.2% | **Rất ổn định**, Entity encoding giúp nhieu |
| SVR | 0.9604 | 0.9562 | 0.4% | **Rất ổn định** |
| XGBoost | 0.9779 | 0.7405 | **23.7%** | **SUY GIAM MẠNH** |

**Nhận xét Panel Data**:
- LR va SVR: Suy giảm **rất nhẹ** (<0.5%) -> Entity encoding (Fixed Effects) giúp model hoc đặc điểm riêng của từng quốc gia
- XGBoost: Vẫn suy giảm **~24%** -> **Vấn đề cơ bản cua Tree-based models**

### 5.3 Phát hiện quan trọng - TẠI SAO XGBOOST SUY GIAM MẠNH?

**XGBoost giảm ~24% o CA HAI cách tiếp cận (Pooled va Panel)**:

```
NGUYÊN NHÂN GỐC:
- XGBoost (Tree-based) học bằng cach tạo các "ngưỡng" (thresholds)
- Ví dụ: Nếu GDP > 40,000 -> CO2 = 500,000
- Cac ngưỡng nay chỉ hoạt động trong PHẠM VI dữ liệu training

VAN DE VOI TIME-SERIES:
- Training: 2001-2014 -> GDP tu 1,000 den 50,000
- Test: 2015-2019 -> GDP có thể len 60,000 (NGOAI PHAM VI)
- XGBoost khong the dự đoán cho GDP = 60,000 vi chưa thấy bao giờ!
- Kết quả: Du doan = giá trị cao nhất đã thấy (50,000) -> SAI
```

**TẠI SAO LR VA SVR KHONG BI?**

```
LINEAR REGRESSION:
- Học công thức: CO2 = 10 * GDP + 5 * Energy + ...
- Với GDP = 60,000: CO2 = 10 * 60,000 = 600,000
- Co the tính cho BẤT KỲ giá trị nào -> NGOẠI SUY TỐT

SVR (RBF Kernel):
- RBF kernel tao "vùng ảnh hưởng" xung quanh cac điểm training
- Với dữ liệu gan pham vì training, van có thể nội suy/ngoại suy
- Suy giảm nhẹ vì test data (2015-2019) khong quá xa training (2001-2014)
```

### 5.4 So sánh Pooled vs Panel

| Metric | Pooled Data | Panel Data | Nhận xét |
|--------|-------------|------------|----------|
| LR R2 (TS) | 0.8310 | **0.9934** | Panel tốt hon 16% |
| SVR R2 (TS) | 0.9028 | **0.9562** | Panel tốt hon 5% |
| XGBoost R2 (TS) | 0.7369 | 0.7405 | Tương đương |

**Kết luận**: Panel Data (Entity encoding) giúp LR va SVR tốt hon đáng kể, nhưng **KHÔNG cứu được XGBoost**.

### 5.5 Bẫy nội suy (Interpolation Trap)

**Random Split**:
- Tron lan cac năm -> Model chỉ cần "nội suy" giữa cac diem da biết
- XGBoost R2 = 0.97+ (rat cao) vi chỉ cần "điền vào chỗ trống"

**Time-Series Split**:
- Train tren quá khứ (2001-2014), test tren tương lai (2015-2019)
- Model phải "ngoại suy" -> XGBoost thất bại (R2 giảm 24%)

### 5.6 Khuyến nghị

1. **PHẢI dùng Time-Series Split** cho bài toán dự báo chuoi thời gian
2. **Loại bỏ CO2_lag1** khi dùng Random Split để tránh Data Leakage
3. **Linear Regression + Panel Data** la lựa chọn tốt nhat cho forecasting (R2 = 0.9934)
4. **XGBoost KHÔNG phù hợp** cho forecasting độc lập, nhưng có thể dung trong **Hybrid Model** (LR + XGBoost)
5. **SVR** la lựa chọn trung gian tốt (R2 = 0.9562) nếu cần model phi tuyến